 <table>
    <tr>
        <td>
           <img src='https://drive.google.com/uc?id=1V8z6gn4o_8NZqjnGCC4kiaMd10IxOFyF' height='200px' align='center'>
        </td>          
    </tr>
</table>

+ **Alunos:** 
    + Alex Felipe Barbosa
    + Danilo Silva Andrade

---

### 1.) DATA ACQUISITION

---

+ Instalação e Declaração das bibliotecas

In [281]:
# Install libraries first before declaring them
# !pip install pandas
# !pip install numpy
# !pip install plotly

# Declare libraries
import pandas as pd
import numpy as np
import plotly

+ conexão ao database ( temporariamente local )

In [282]:
# Search the path
dir_db = ('C:\\Users\\da04257\\Downloads\\tcc\\dir_db\\')

# Atributte the path database and search file
archive = ('dir_db//DadosEquipamentos_v3.csv')

+ leitura inicial da base de dados

In [283]:
# Read and select columns
dataset = pd.read_csv(archive, sep=';')

# Days of harvest
days_eqpto = dataset.groupby('CD_EQUIPAMENTO').DT_APONTAMENTO.nunique() # 264 e 257 ofic.
days = (days_eqpto / 2)

# Hours of day
hours = 24

# Hours of harvest
hh = (days.mean() * hours)

# Print first 3 rows of dataset
dataset.head(3)

,DT_APONTAMENTO,ANO,MES,CD_EQUIPAMENTO,CD_ATIVIDADE,NM_ATIVIDADE,FG_ATIVIDADE_PRODUTIVA,FG_ATIVIDADE_MANUTENCAO,VA_DURACAO,VA_TEMPO_MOTOR_LIGADO,VA_TEMPO_MOTOR_OCIOSO,VA_TEMPO_TRABALHO,VA_ABASTECIMENTO,VA_KM_TOTAL,VA_KM_TRABALHO,VA_VELOCIDADE_MEDIA_TR,VA_AREA_TRABALHO
0,25/03/2019,2019,3,225049,2001,Arremate,1,0,2818,2770,12,116,0,"0,85895","0,09467","2,93824","0,01419"
1,25/03/2019,2019,3,225049,2000,Manobra Colhedora,1,0,562,554,0,1,0,"0,21556","0,00065","2,32143","0,0001"
2,25/03/2019,2019,3,225049,1942,Maquina Embuchada,1,0,11,11,0,0,0,"0,00556",0,0,0


---

### 2.) DATA PREPARATION

---

##### 2.1) Análise Descritiva

In [284]:
# Display the following:
# - type of database
# - size of dataframe and yours columns
# - datatypes the columns
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370980 entries, 0 to 370979
Data columns (total 17 columns):
DT_APONTAMENTO             370980 non-null object
ANO                        370980 non-null int64
MES                        370980 non-null int64
CD_EQUIPAMENTO             370980 non-null int64
CD_ATIVIDADE               370980 non-null int64
NM_ATIVIDADE               370720 non-null object
FG_ATIVIDADE_PRODUTIVA     370980 non-null int64
FG_ATIVIDADE_MANUTENCAO    370980 non-null int64
VA_DURACAO                 370980 non-null int64
VA_TEMPO_MOTOR_LIGADO      370980 non-null int64
VA_TEMPO_MOTOR_OCIOSO      370980 non-null int64
VA_TEMPO_TRABALHO          370980 non-null int64
VA_ABASTECIMENTO           370980 non-null object
VA_KM_TOTAL                370980 non-null object
VA_KM_TRABALHO             370980 non-null object
VA_VELOCIDADE_MEDIA_TR     370980 non-null object
VA_AREA_TRABALHO           370980 non-null object
dtypes: int64(10), object(7)
memory usage: 48.1+ 

In [285]:
# Display the following:
# - main statistical measures of the dataframe
dataset.describe()

,ANO,MES,CD_EQUIPAMENTO,CD_ATIVIDADE,FG_ATIVIDADE_PRODUTIVA,FG_ATIVIDADE_MANUTENCAO,VA_DURACAO,VA_TEMPO_MOTOR_LIGADO,VA_TEMPO_MOTOR_OCIOSO,VA_TEMPO_TRABALHO
count,370980.000000,370980.000000,370980.000000,370980.000000,370980.000000,370980.000000,370980.000000,370980.000000,370980.00000,370980.000000
mean,2019.549043,7.149811,225583.483153,1862.182719,0.486687,0.136199,3071.159682,1778.561502,7.97712,1097.049232
std,0.497590,2.433846,265.895636,268.891344,0.499823,0.343000,7146.479350,3897.042482,61.26452,3659.962925
min,2019.000000,3.000000,225049.000000,1000.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000
25%,2019.000000,5.000000,225395.000000,1905.000000,0.000000,0.000000,82.000000,7.000000,0.00000,0.000000
50%,2020.000000,7.000000,225724.000000,1943.000000,0.000000,0.000000,671.000000,80.000000,0.00000,0.000000
75%,2020.000000,9.000000,225880.000000,2000.000000,1.000000,0.000000,2896.000000,1290.000000,0.00000,16.000000
max,2020.000000,12.000000,225885.000000,2002.000000,1.000000,1.000000,89055.000000,41168.000000,8465.00000,40484.000000


##### 2.2) Transformação e Limpeza dos dados
+ Transformar, renomear, agrupar, etc.

In [286]:
# data grouping and transformation in hours
dt_group = dataset.groupby([dataset['DT_APONTAMENTO'], 'ANO', 'MES', 'CD_EQUIPAMENTO', 'CD_ATIVIDADE', 'NM_ATIVIDADE', 'FG_ATIVIDADE_PRODUTIVA', 'FG_ATIVIDADE_MANUTENCAO'])[['VA_TEMPO_MOTOR_LIGADO', 'VA_TEMPO_MOTOR_OCIOSO', 'VA_DURACAO']].sum() / 60 / 60

# rounding columns dataset
dt_group = dt_group.round(2)

# Display first 5 rows of dataset update
# dt_group.head(5)
# ------ For Check: db_resumo.query('DT_APONTAMENTO == "07/03/2019"')

# Saving the dataset for Day / Equipto / Atividade
dt_group.to_csv('dir_db\dt_group.csv', encoding='utf-8', index=True)

# Search the database
archive = ('dir_db/dt_group.csv')

# Read and select columns
dataset = pd.read_csv(archive, sep=',')

# Print first 10 rows of dataset
dataset.head(5)

# Decode Activities
dataset['FG_ATIVIDADE_PRODUTIVA'] = dataset.FG_ATIVIDADE_PRODUTIVA.replace(0,"Improdutiva")
dataset['FG_ATIVIDADE_PRODUTIVA'] = dataset.FG_ATIVIDADE_PRODUTIVA.replace(1,"Produtiva")






### 6. Agrupando diversos datasets: por Ano / Mes / Atividade

dt_year_month = dataset.groupby([dataset['ANO'], dataset['MES'], 'CD_ATIVIDADE', 'NM_ATIVIDADE', 'FG_ATIVIDADE_PRODUTIVA', 'FG_ATIVIDADE_MANUTENCAO'])[['VA_TEMPO_MOTOR_LIGADO', 'VA_TEMPO_MOTOR_OCIOSO', 'VA_DURACAO']].sum()
# rounding columns dataset
dt_year_month = dt_year_month.round(2)


# Groupby Year
dt_year = dataset.groupby([dataset['ANO'], 'CD_ATIVIDADE', 'NM_ATIVIDADE', 'FG_ATIVIDADE_PRODUTIVA', 'FG_ATIVIDADE_MANUTENCAO'])[['VA_DURACAO']].sum()
# rename column of database
#dt_year = dt_year.rename(columns={"FG_ATIVIDADE_PRODUTIVA": "TIPO_ATIVIDADE"})
# rounding columns dataset
dt_year = dt_year.round(2)

# Saving the datasets
dt_year_month.to_csv('dir_db\\dt_year_month.csv', encoding='utf-8', index=True)
dt_year.to_csv('dir_db\\dt_year.csv', encoding='utf-8', index=True)

# Search the database
archive_year_month = ('dir_db/dt_year_month.csv')
archive_year = ('dir_db/dt_year.csv')

# Read and select columns
dt_year_month = pd.read_csv(archive_year_month, sep=',')
dt_year = pd.read_csv(archive_year, sep=',')

# Define sum hours 2019
horas_tot_19 = dt_year.query('ANO == 2019')
horas_tot_19 = (horas_tot_19['VA_DURACAO'].sum() * 1)

# Define sum hours 2020
horas_tot_20 = dt_year.query('ANO == 2020')
horas_tot_20 = (horas_tot_20['VA_DURACAO'].sum() * 1)

dt_year['%_HORAS_ATV'] = np.where(dt_year['ANO'] == 2019, round(dt_year['VA_DURACAO'] / horas_tot_19 * 100,2) , round(dt_year['VA_DURACAO'] / horas_tot_20 * 100,2))


In [287]:
import plotly.figure_factory as ff

# Rename column of database
dt_year_graf = dt_year.rename(columns={"VA_DURACAO": "HORAS_ATV", "FG_ATIVIDADE_PRODUTIVA": "TIPO_ATIVIDADE"}, errors="raise")

# Select columns of database
dt_year_graf = dt_year_graf[['ANO', 'TIPO_ATIVIDADE', 'NM_ATIVIDADE', '%_HORAS_ATV']]

# Sort Values in database
dt_year_graf = dt_year_graf.sort_values(['ANO', 'TIPO_ATIVIDADE', '%_HORAS_ATV'], ascending=False)

# Plot database
fig =  ff.create_table(dt_year_graf, height_constant=20)
print(' / ')
print(' / ')
print(' / Relação % de Horas Produtivas e Improdutivas por Ano e Tipos de Atividades')
fig.show()

 / 
 / 
 / Relação % de Horas Produtivas e Improdutivas por Ano e Tipos de Atividades


In [ ]:
# rename column of database
dataset = dataset.rename(columns={"VA_DURACAO": "HORAS_ATV", "FG_ATIVIDADE_PRODUTIVA": "TIPO_ATIVIDADE"}, errors="raise") #V_TEMPO_PROD_MED_DIA_ATIVIDADE

---

### 3. Explanatory Analysis

---

In [316]:
########## FATOR CLIMÁTICO ##########

atv_1 = dt_year.query("CD_ATIVIDADE == 1910")  # Chuva / Escesso de Umidade
atv_2 = dt_year.query("CD_ATIVIDADE == 1921")  # Outros Fatores Climáticos
atv_3 = dt_year.query("CD_ATIVIDADE == 1935")  # Incêndio

dt_clima = pd.concat([atv_1, atv_2, atv_3], ignore_index=True)
dt_clima['CD_ATIVIDADE'] = "Climatica"
dt_clima['%_ATV'] = np.where(dt_clima['ANO'] == 2019, round(dt_clima['VA_DURACAO'] / horas_tot_19 * 100,2) , round(dt_clima['VA_DURACAO'] / horas_tot_20 * 100,2))

dt_clima = dt_clima.groupby([dt_clima['ANO'], 'CD_ATIVIDADE'])[['VA_DURACAO', '%_ATV']].sum()

# Saving the datasets
dt_clima.to_csv('dir_db\\dt_clima.csv', encoding='utf-8', index=True)

# import
archive_dt_clima = ('dir_db/dt_clima.csv')
# read
dt_clima = pd.read_csv(archive_dt_clima, sep=',')
dt_clima = dt_clima.drop(['VA_DURACAO'], axis=1)

# Insert news columns in dataframe 
dt_clima['HC'] = hh
dt_clima['DIAS'] = 0
dt_clima['DIF_DIAS'] = 0

# Calculate days and dif_days
dt_clima['DIAS'] = round(dt_clima['%_ATV'] * dt_clima['HC'] / 24 / 100,0)
dt_clima['DIF_DIAS'] = dt_clima['DIAS'][0] - dt_clima['DIAS'][1]





########## FATOR MANUTENÇÃO ###########

atv_1 = dt_year.query("FG_ATIVIDADE_MANUTENCAO == 1")  # Todas paradas por manutenção

dt_manut = pd.concat([atv_1], ignore_index=True)
dt_manut['CD_ATIVIDADE'] = "Manutencao"
dt_manut['%_ATV'] = np.where(dt_manut['ANO'] == 2019, round(dt_manut['VA_DURACAO'] / horas_tot_19 * 100,2) , round(dt_manut['VA_DURACAO'] / horas_tot_20 * 100,2))

dt_manut = dt_manut.groupby([dt_manut['ANO'], 'CD_ATIVIDADE'])[['VA_DURACAO', '%_ATV']].sum()

# Saving the datasets
dt_manut.to_csv('dir_db\\dt_manut.csv', encoding='utf-8', index=True)

# import
archive_dt_manut = ('dir_db/dt_manut.csv')
# read
dt_manut = pd.read_csv(archive_dt_manut, sep=',')
dt_manut = dt_manut.drop(['VA_DURACAO'], axis=1)

# Insert news columns in dataframe
dt_manut['HC'] = hh
dt_manut['DIAS'] = 0
dt_manut['DIF_DIAS'] = 0

# Calculate days and dif_days
dt_manut['DIAS'] = round(dt_manut['%_ATV'] * dt_manut['HC'] / 24 / 100,0)
dt_manut['DIF_DIAS'] = dt_manut['DIAS'][0] - dt_manut['DIAS'][1]





########## FATOR OPERACIONAL ##########

atv_1 = dt_year.query("FG_ATIVIDADE_MANUTENCAO == 0 and FG_ATIVIDADE_PRODUTIVA == 'Improdutiva'")  # Todas paradas diferentes de manutenção e improdutivas
atv_2 = atv_1.query("CD_ATIVIDADE != 1910")  # Chuva / Escesso de Umidade
atv_3 = atv_2.query("CD_ATIVIDADE != 1921")  # Outros Fatores Climáticos
atv_4 = atv_3.query("CD_ATIVIDADE != 1935")  # Incêndio

dt_oper = pd.concat([atv_4], ignore_index=True)
dt_oper['CD_ATIVIDADE'] = "Operacional"
dt_oper['%_ATV'] = np.where(dt_oper['ANO'] == 2019, round(dt_oper['VA_DURACAO'] / horas_tot_19 * 100,2) , round(dt_oper['VA_DURACAO'] / horas_tot_20 * 100,2))

dt_oper = dt_oper.groupby([dt_oper['ANO'], 'CD_ATIVIDADE'])[['VA_DURACAO', '%_ATV']].sum()

# Saving the datasets
dt_oper.to_csv('dir_db\\dt_oper.csv', encoding='utf-8', index=True)

# import
archive_dt_oper = ('dir_db/dt_oper.csv')
# read
dt_oper = pd.read_csv(archive_dt_oper, sep=',')
dt_oper = dt_oper.drop(['VA_DURACAO'], axis=1)

# Insert news columns in dataframe
dt_oper['HC'] = hh
dt_oper['DIAS'] = 0
dt_oper['DIF_DIAS'] = 0

# Calculate days and dif_days
dt_oper['DIAS'] = round(dt_oper['%_ATV'] * dt_oper['HC'] / 24 / 100,0)
dt_oper['DIF_DIAS'] = dt_oper['DIAS'][0] - dt_oper['DIAS'][1]




########## FATOR PRODUTIVO ##########

dt_produtivo = dt_year.query("FG_ATIVIDADE_MANUTENCAO == 0 and FG_ATIVIDADE_PRODUTIVA == 'Produtiva'")  # Todas paradas diferentes de manutenção e improdutivas

dt_produtivo = pd.concat([dt_produtivo], ignore_index=True)

dt_produtivo['CD_ATIVIDADE'] = "Produtiva"
dt_produtivo['%_ATV'] = np.where(dt_produtivo['ANO'] == 2019, round(dt_produtivo['VA_DURACAO'] / horas_tot_19 * 100,2) , round(dt_produtivo['VA_DURACAO'] / horas_tot_20 * 100,2))

dt_produtivo = dt_produtivo.groupby([dt_produtivo['ANO'], 'CD_ATIVIDADE'])[['VA_DURACAO', '%_ATV']].sum()

# Saving the datasets
dt_produtivo.to_csv('dir_db\\dt_produtivo.csv', encoding='utf-8', index=True)

# import
archive_dt_produtivo = ('dir_db/dt_produtivo.csv')
# read
dt_produtivo = pd.read_csv(archive_dt_produtivo, sep=',')
dt_produtivo = dt_produtivo.drop(['VA_DURACAO'], axis=1)

# Insert news columns in dataframe
dt_produtivo['HC'] = hh
dt_produtivo['DIAS'] = 0
dt_produtivo['DIF_DIAS'] = 0

# Calculate days and dif_days
dt_produtivo['DIAS'] = round(dt_produtivo['%_ATV'] * dt_produtivo['HC'] / 24 / 100,0)
#dt_produtivo['DIF_DIAS'] = dt_produtivo['DIAS'][0] - dt_produtivo['DIAS'][1]

# Concat dataframe (Oficial Result)
df_result = pd.concat([dt_clima,dt_manut,dt_oper,dt_produtivo])
df_result.drop(columns=['HC'])

# rename column of database
df_result = df_result.rename(columns={"CD_ATIVIDADE": "ATIVIDADE"})



In [319]:
import plotly.figure_factory as ff

df_result = df_result[['ANO', 'ATIVIDADE', '%_ATV', 'DIAS', 'DIF_DIAS']]
df_result['%_ATV'] = round(df_result['%_ATV'],2)
fig =  ff.create_table(df_result, height_constant=20)
print('Análise das paradas por classe de Atividade')
fig.show()

Análise das paradas por classe de Atividade


---

### 4.) VISUALIZATION AND COMMUNICATION

---

##### 4.1) ATIVIDADE IMPRODUTIVAS

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1910")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA != 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas em Chuva - <b>Atividade 1910</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1905")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA != 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas em Manutenção - <b>Atividade 1905</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1939")
#df_eqpto = df_eqpto.query("CD_EQUIPAMENTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA != 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas Aguardando Transbordo - <b>Atividade 1939</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1924")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA != 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas em Pátio - <b>Atividade 1924</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1903")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA != 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas Aguardando Caminhão - <b>Atividade 903</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1964")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA != 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas na Fila Indústria - <b>Atividade 1964</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1943")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA != 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas Lavando / Lubrificando - <b>Atividade 1943</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1970")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA != 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas em Deslocamento - <b>Atividade 1970</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1933")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA != 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas em Troca de Facas - <b>Atividade 1933</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1917")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA != 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2019.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, text=df2020.V_TEMPO_PROD_MED_DIA_ATIVIDADE, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas em Mudança Prancha - <b>Atividade 1917</b>")
fig.show()

##### 4.2) ATIVIDADE PRODUTIVAS

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1130")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA == 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.VA_TEMPO_MOTOR_LIGADO, text=df2019.VA_TEMPO_MOTOR_LIGADO, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.VA_TEMPO_MOTOR_LIGADO, text=df2020.VA_TEMPO_MOTOR_LIGADO, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas de Corte Mecânico - <b>Atividade 1130</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 2000")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA == 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.VA_TEMPO_MOTOR_LIGADO, text=df2019.VA_TEMPO_MOTOR_LIGADO, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.VA_TEMPO_MOTOR_LIGADO, text=df2020.VA_TEMPO_MOTOR_LIGADO, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas em Manobra - <b>Atividade 2000</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 2001")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA == 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.VA_TEMPO_MOTOR_LIGADO, text=df2019.VA_TEMPO_MOTOR_LIGADO, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.VA_TEMPO_MOTOR_LIGADO, text=df2020.VA_TEMPO_MOTOR_LIGADO, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas de Arremate - <b>Atividade 2001</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 2002")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA == 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.VA_TEMPO_MOTOR_LIGADO, text=df2019.VA_TEMPO_MOTOR_LIGADO, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.VA_TEMPO_MOTOR_LIGADO, text=df2020.VA_TEMPO_MOTOR_LIGADO, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Tempo de Esvaziamento(h) - <b>Atividade 2002</b>")
fig.show()

In [ ]:
import plotly.graph_objects as go

df_eqpto = dt_year_month.query("CD_ATIVIDADE == 1942")
#df_eqpto = df_eqpto.query("EQUIPTO == 1")
df_eqpto = df_eqpto.query("FG_ATIVIDADE_PRODUTIVA == 1")

df2019 = df_eqpto.query('ANO=="2019"')
df2020 = df_eqpto.query('ANO=="2020"')

fig = go.Figure(data=[   
    go.Bar(name='2019', x=df2019.MES, y=df2019.VA_TEMPO_MOTOR_LIGADO, text=df2019.VA_TEMPO_MOTOR_LIGADO, textposition='auto'),
    go.Bar(name='2020', x=df2020.MES, y=df2020.VA_TEMPO_MOTOR_LIGADO, text=df2020.VA_TEMPO_MOTOR_LIGADO, textposition='auto')
])

#marker_color='crimson'
fig.update_traces(marker_line_color='rgb(255,255,255)', marker_line_width=0.5, opacity=0.9)
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', title="Horas de Desembuchamento - <b>Atividade 1942</b>")
fig.show()

In [ ]:
dt_clima['DIAS'] = 0
dt_clima['DIF_DIAS'] = 0

dt_clima1 = dt_clima.loc[(dt_clima['ANO']) == 2019]
dt_clima1.loc[0,'DIAS'] = round((dt_clima1.iloc[-1,2] * hh) / 24 / 100,0)

dt_clima2 = dt_clima.loc[(dt_clima['ANO']) == 2020]
dt_clima2.loc[1,'DIAS'] = round((dt_clima2.iloc[-1,2] * hh) / 24 / 100,0)

dt_clima = pd.concat([dt_clima1,dt_clima2])
dt_clima['DIF_DIAS'] = dt_clima.DIAS[0] - dt_clima.DIAS[1]
dt_clima